# Felipe Castillo
# Loan Approval Data
# Data Mining
# 05/14/2022

In [1]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import r2_score ,accuracy_score,mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import pandas as pd
import os
import numpy as np
import sys

#week 10
os.chdir('C:\DataScience_DSC_550\Week9')

In [197]:
#!{sys.executable} -m pip install scikit-learn
#!{sys.executable} -m pip install -U scikit-learn

# 1.Import the dataset and ensure that it loaded properly.

In [2]:
#loading data
LAD_DF = pd.read_csv("Loan_Train.csv")

In [3]:
LAD_DF.head(3)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y


# 2.Prepare the data for modeling by performing the following steps:

In [4]:
#1.Drop the column “Load_ID.”
LAD_DF = LAD_DF.drop(["Loan_ID"], axis=1)

In [5]:
LAD_DF.head(3)

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y


In [6]:
LAD_DF.shape

(614, 12)

In [7]:
#Check 
#seeing null values
LAD_DF.isnull().sum()

Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [8]:
#2.Drop any rows with missing data.
#starting with row one and dropping if there is a null value
LAD_DF.dropna(
    axis=0,
    how='any',
    thresh=None,
    subset=None,
    inplace=True
)

In [9]:
LAD_DF.head(3)

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y


In [10]:
#New shape Shows 12 columns and 480 rows
#VS the first, the data before the drop that shows 12 columns and 614 
LAD_DF.shape

(480, 12)

In [12]:
#Now there are no nulls 
LAD_DF.isnull().sum()

Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [231]:
#3.Convert the categorical features into dummy variables.

In [13]:
#Convert data to categorical features
LAD_DF_Dum = pd.get_dummies(LAD_DF)

In [14]:
LAD_DF_Dum.head(5)

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Female,Gender_Male,Married_No,Married_Yes,Dependents_0,...,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Loan_Status_N,Loan_Status_Y
1,4583,1508.0,128.0,360.0,1.0,0,1,0,1,0,...,0,1,0,1,0,1,0,0,1,0
2,3000,0.0,66.0,360.0,1.0,0,1,0,1,1,...,0,1,0,0,1,0,0,1,0,1
3,2583,2358.0,120.0,360.0,1.0,0,1,0,1,1,...,0,0,1,1,0,0,0,1,0,1
4,6000,0.0,141.0,360.0,1.0,0,1,1,0,1,...,0,1,0,1,0,0,0,1,0,1
5,5417,4196.0,267.0,360.0,1.0,0,1,0,1,0,...,0,1,0,0,1,0,0,1,0,1


# 3. Split the data into a training and test set, where the “Loan_Status” column is the target.

In [15]:
#Setting loan status as target
target = LAD_DF_Dum[["Loan_Status_N","Loan_Status_Y"]]
#Setting the other features  and dropping the targets
features = LAD_DF_Dum.drop(["Loan_Status_N","Loan_Status_Y"], axis=1).copy()

In [16]:
#splitting and trainning
features_test , features_train, target_test, target_train = train_test_split(features,target,test_size = 0.20)

# 4. Create a pipeline with a min-max scaler and a KNN classifier (see section 15.3 in the Machine Learning with Python Cookbook).

In [17]:
#instantiating minmaxscaler class
scaler = MinMaxScaler()

In [18]:
#instantiating KNeighborsClassifier
knn = KNeighborsClassifier()

In [19]:
#Setting scaler and KNeighborsClassifier
pipe = Pipeline([("scaler",scaler), ("knn",knn)])

# 5. Fit a default KNN classifier to the data with this pipeline. Report the model accuracy on the test set. Note: Fitting a pipeline model works just like fitting a regular model.

In [20]:
#fitting model
model  = pipe.fit(features_train,target_train)

In [21]:
#passing test features
target_predict = model.predict(features_test)

In [22]:
#getting accuaracy_score 
accuracy_score(target_test,target_predict )

0.7083333333333334

# 6.Create a search space for your KNN classifier where your “n_neighbors” parameter varies from 1 to 10. (see section 15.3 in the Machine Learning with Python Cookbook).

In [23]:
#knn__n_neighbors as a parameter
search_space = [{"knn__n_neighbors" : [1,2,3,4,5,6,7,8,9,10]}]

# 7. Fit a grid search with your pipeline, search space, and 5-fold cross-validation to find the best value for the “n_neighbors” parameter.

In [24]:
#Passing pipe ,and search space
grid= GridSearchCV(pipe, search_space, cv=5, verbose=0)

# 8.Find the accuracy of the grid search best model on the test set. Note: It is possible that this will not be an improvement over the default model, but likely it will be.

In [25]:
#available parameters
grid.get_params().keys()

dict_keys(['cv', 'error_score', 'estimator__memory', 'estimator__steps', 'estimator__verbose', 'estimator__scaler', 'estimator__knn', 'estimator__scaler__clip', 'estimator__scaler__copy', 'estimator__scaler__feature_range', 'estimator__knn__algorithm', 'estimator__knn__leaf_size', 'estimator__knn__metric', 'estimator__knn__metric_params', 'estimator__knn__n_jobs', 'estimator__knn__n_neighbors', 'estimator__knn__p', 'estimator__knn__weights', 'estimator', 'n_jobs', 'param_grid', 'pre_dispatch', 'refit', 'return_train_score', 'scoring', 'verbose'])

In [26]:
#fitting train and features and target train 
grid.fit(features_train,target_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('knn', KNeighborsClassifier())]),
             param_grid=[{'knn__n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}])

In [27]:
grid_prediction = grid.predict(features_test)

In [28]:
#Getting accuracy
accuracy_score(target_test,grid_prediction )

0.7109375

# 9.Now, repeat steps 6 and 7 with the same pipeline, but expand your search space to include logistic regression and random forest models with the hyperparameter values in section 12.3 of the Machine Learning with Python Cookbook.

In [29]:
pipe_LD = Pipeline([("classifier",RandomForestClassifier())])

In [30]:
#All feilds that are passed a from page 12.3 
#passing both models
search_space_LD =[{"classifier": [LogisticRegression(solver='liblinear')],
                  "classifier__penalty": ['l1','l2'],
                  "classifier__C": np.logspace(0,4,10)},
                 {"classifier": [RandomForestClassifier()],
                 "classifier__n_estimators": [10,100,1000],
                 "classifier__max_features": [1,2,3]}]

In [31]:
grid_LR = GridSearchCV(pipe_LD, search_space_LD, cv=5, verbose=0)

# 10.What are the best model and hyperparameters found in the grid search? Find the accuracy of this model on the test set.

In [32]:
#available keys 
grid_LR.get_params().keys()

dict_keys(['cv', 'error_score', 'estimator__memory', 'estimator__steps', 'estimator__verbose', 'estimator__classifier', 'estimator__classifier__bootstrap', 'estimator__classifier__ccp_alpha', 'estimator__classifier__class_weight', 'estimator__classifier__criterion', 'estimator__classifier__max_depth', 'estimator__classifier__max_features', 'estimator__classifier__max_leaf_nodes', 'estimator__classifier__max_samples', 'estimator__classifier__min_impurity_decrease', 'estimator__classifier__min_samples_leaf', 'estimator__classifier__min_samples_split', 'estimator__classifier__min_weight_fraction_leaf', 'estimator__classifier__n_estimators', 'estimator__classifier__n_jobs', 'estimator__classifier__oob_score', 'estimator__classifier__random_state', 'estimator__classifier__verbose', 'estimator__classifier__warm_start', 'estimator', 'n_jobs', 'param_grid', 'pre_dispatch', 'refit', 'return_train_score', 'scoring', 'verbose'])

In [33]:
best_model = grid_LR.fit(features_train,target_train)

C:\Users\cast6\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
100 fits failed out of a total of 145.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\cast6\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\cast6\anaconda3\lib\site-packages\sklearn\pipeline.py", line 382, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\cast6\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1141, in fit
    X, y = self._validate_data(
  File "C:\Users\cas

In [34]:
best_model.best_estimator_.get_params()["classifier"]


RandomForestClassifier(max_features=3, n_estimators=1000)

In [38]:
#best models is RandomForestClassifier
best_model.best_estimator_.get_params()

{'memory': None,
 'steps': [('classifier',
   RandomForestClassifier(max_features=3, n_estimators=1000))],
 'verbose': False,
 'classifier': RandomForestClassifier(max_features=3, n_estimators=1000),
 'classifier__bootstrap': True,
 'classifier__ccp_alpha': 0.0,
 'classifier__class_weight': None,
 'classifier__criterion': 'gini',
 'classifier__max_depth': None,
 'classifier__max_features': 3,
 'classifier__max_leaf_nodes': None,
 'classifier__max_samples': None,
 'classifier__min_impurity_decrease': 0.0,
 'classifier__min_samples_leaf': 1,
 'classifier__min_samples_split': 2,
 'classifier__min_weight_fraction_leaf': 0.0,
 'classifier__n_estimators': 1000,
 'classifier__n_jobs': None,
 'classifier__oob_score': False,
 'classifier__random_state': None,
 'classifier__verbose': 0,
 'classifier__warm_start': False}

In [39]:
bestM_prediction = best_model.predict(features_test)

In [40]:
#So far accuaracy is the highest
accuracy_score(target_test,bestM_prediction )

0.7473958333333334

# 11. Summarize your results.



1. 0.70 without grid 

2. 0.71 with grid

3. 0.74 best models

When I first ran the model with it was at 70 percent.  When I added the cross-validation grid, I was able to get the mode to 71 percent. There was a slight change between, but the change is positive.  I added both models and the one that came up was Random Forest Classifier. The accuracy was a 74 percent. Meaning that the model was better than K Neighbors Classifier. From 70 this model was the better choice.





